In [18]:
import pandas as pd
import numpy as np
from datetime import datetime, date
from sweref99 import projections
import folium
from folium import plugins
from selenium import webdriver
import matplotlib.pyplot as plt
from sklearn import linear_model, datasets
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display, clear_output

In [3]:
path='./data/Insatser till brand i skog och mark 2000-2020.xlsx'

In [4]:
#tm is used for the conversions of easting and northing to longitude and latitude
tm = projections.make_transverse_mercator("SWEREF_99_TM")

In [8]:
df = pd.read_excel(path)
sample = df.sample(50)

In [6]:
for column in df:
    if df[column].isnull().any():
       print('{0} has {1} null values'.format(column, df[column].isnull().sum()))

sweref99Norr has 20718 null values
sweref99Ost has 20717 null values


In [7]:
#Function that calculates number of missing data in column of dataframe and prints result.
def missing(df,column):
       x = len(df)
       if df[column].isnull().any():
           print('{0} has total of {1} null values'.format(column, df[column].isnull().sum()))
           print ('In the column {0}'.format(column), round(df[column].count()-1/x * 100, 3), '% of the cells have missing values')
 

In [9]:
#Missing values of column 'Gras'
missing(sample,'sweref99Norr')
missing(sample,'sweref99Ost')
sample.isnull().sum(axis = 0)

sweref99Norr has total of 10 null values
In the column sweref99Norr 38.0 % of the cells have missing values
sweref99Ost has total of 10 null values
In the column sweref99Ost 38.0 % of the cells have missing values


ar                               0
datum                            0
tid                              0
kommun                           0
kommunKortNamn                   0
verksamhetText                   0
sweref99Norr                    10
sweref99Ost                     10
BEJBbrandorsakText               0
arealProduktivSkogsmark_m2       0
arealAnnanTradbevuxenMark_m2     0
arealMarkUtanTrad_m2             0
dtype: int64

In [14]:
sample['TotArea'] = sample['arealProduktivSkogsmark_m2']+sample['arealAnnanTradbevuxenMark_m2']+sample['arealMarkUtanTrad_m2']
sample

,ar,datum,tid,kommun,kommunKortNamn,verksamhetText,sweref99Norr,sweref99Ost,BEJBbrandorsakText,arealProduktivSkogsmark_m2,arealAnnanTradbevuxenMark_m2,arealMarkUtanTrad_m2,Latitude,Longitude,TotArea
80185,2017,2017-06-28,15:40:00,1287,Trelleborg,Verksamhet inte knuten till en byggnad,6137248.0,394174.0,Okänd,0,0,5,55.370117,13.330166,5
38140,2007,2007-04-05,21:25:00,488,Trosa,Verksamhet inte knuten till en byggnad,6534563.0,637926.0,Okänd,0,30,0,58.928516,17.396067,30
99353,2020,2020-08-17,11:36:00,1286,Ystad,Verksamhet inte knuten till en byggnad,6151246.0,432963.0,Okänd,0,0,100,55.502705,13.938700,100
92342,2019,2019-05-20,21:10:00,180,Stockholm,Verksamhet inte knuten till en byggnad,6588315.0,665978.0,Avsiktlig brand,0,100,150,59.400675,17.923612,250
69487,2015,2015-04-24,16:00:00,2180,Gävle,Verksamhet inte knuten till en byggnad,6727339.0,612347.0,Okänd,0,0,330,60.665962,17.055877,330
18459,2003,2003-04-16,18:06:00,484,Eskilstuna,Verksamhet inte knuten till en byggnad,6582215.0,582241.0,Okänd,0,3,0,59.370614,16.447120,3
12905,2002,2002-06-11,11:39:00,127,Botkyrka,Verksamhet inte knuten till en byggnad,6562198.0,662472.0,Okänd,15000,0,0,59.167796,17.842337,15000
86082,2018,2018-07-06,21:54:00,882,Oskarshamn,Verksamhet inte knuten till en byggnad,6375287.0,590788.0,Blixtnedslag,1000,500,80000,57.511034,16.515411,81500
51240,2010,2010-04-19,09:11:00,1980,Västerås,Verksamhet inte knuten till en byggnad,6608677.0,589423.0,Avsiktlig brand,0,0,1,59.606689,16.584534,1
92952,2019,2019-06-01,22:46:00,1283,Helsingborg,Verksamhet inte knuten till en byggnad,6214873.0,355714.0,Avsiktlig brand,0,1,0,56.057256,12.682867,1


In [15]:
#Functions for converting easting and northing to latitudes and longitudes.
def toLat(E,N):
    lat, lon = tm.grid_to_geodetic(N,E)
    return lat
def toLon(E,N):
    lat, lon = tm.grid_to_geodetic(N,E)
    return lon

In [16]:
#Applying functions to create to new columns, Longitude and Latitude for reported fires.
sample['Latitude'] = sample.apply(lambda row: toLat(row['sweref99Ost'],row['sweref99Norr']),axis=1)
sample['Longitude'] = sample.apply(lambda row: toLon(row['sweref99Ost'],row['sweref99Norr']),axis=1)
sample

,ar,datum,tid,kommun,kommunKortNamn,verksamhetText,sweref99Norr,sweref99Ost,BEJBbrandorsakText,arealProduktivSkogsmark_m2,arealAnnanTradbevuxenMark_m2,arealMarkUtanTrad_m2,Latitude,Longitude,TotArea
80185,2017,2017-06-28,15:40:00,1287,Trelleborg,Verksamhet inte knuten till en byggnad,6137248.0,394174.0,Okänd,0,0,5,55.370117,13.330166,5
38140,2007,2007-04-05,21:25:00,488,Trosa,Verksamhet inte knuten till en byggnad,6534563.0,637926.0,Okänd,0,30,0,58.928516,17.396067,30
99353,2020,2020-08-17,11:36:00,1286,Ystad,Verksamhet inte knuten till en byggnad,6151246.0,432963.0,Okänd,0,0,100,55.502705,13.938700,100
92342,2019,2019-05-20,21:10:00,180,Stockholm,Verksamhet inte knuten till en byggnad,6588315.0,665978.0,Avsiktlig brand,0,100,150,59.400675,17.923612,250
69487,2015,2015-04-24,16:00:00,2180,Gävle,Verksamhet inte knuten till en byggnad,6727339.0,612347.0,Okänd,0,0,330,60.665962,17.055877,330
18459,2003,2003-04-16,18:06:00,484,Eskilstuna,Verksamhet inte knuten till en byggnad,6582215.0,582241.0,Okänd,0,3,0,59.370614,16.447120,3
12905,2002,2002-06-11,11:39:00,127,Botkyrka,Verksamhet inte knuten till en byggnad,6562198.0,662472.0,Okänd,15000,0,0,59.167796,17.842337,15000
86082,2018,2018-07-06,21:54:00,882,Oskarshamn,Verksamhet inte knuten till en byggnad,6375287.0,590788.0,Blixtnedslag,1000,500,80000,57.511034,16.515411,81500
51240,2010,2010-04-19,09:11:00,1980,Västerås,Verksamhet inte knuten till en byggnad,6608677.0,589423.0,Avsiktlig brand,0,0,1,59.606689,16.584534,1
92952,2019,2019-06-01,22:46:00,1283,Helsingborg,Verksamhet inte knuten till en byggnad,6214873.0,355714.0,Avsiktlig brand,0,1,0,56.057256,12.682867,1


In [ ]:
#We need to group fire sizes here but what sizes should we use? What is a "big" fire and what is a "small" fire?

In [ ]:
#Group fires by size
small_wildfires = folium.FeatureGroup(TotArea = '< 10000 Acres')
medium_wildfires = folium.FeatureGroup(TotArea = '10,000 - 50,000 Acres')
large_wildfires = folium.FeatureGroup(TotArea = '50,000 - 100,000 Acres')
xl_wildfires = folium.FeatureGroup(TotArea = '> 100,000 Acres')
